In [ ]:
def make_img(data, obs, params, mode="mean"):
    """
    This function takes physics data in the form [p, x, y, z, theta, phi] and
    the corresponding cluster energy obs and bins it into a n-D image based on
    the information specified by params.

    :param data: physics data in the form [p, x, y, z, theta, phi]
    :param obs: corresponding cluster energy for each physics event
    :param params: dictionary with keys [X, Y, theta, phi], under each key the
    of values for each coordinate is specified. For each coordinate where the
    resolution is specified will be used to create the image, all remaining
    coordinates will be integrated across.
    for example:

    "X_v_theta": {
        "X": {
            "range": (500, 3500),
            "reso": 0.1
        },
        "Y": {
            "range": (500, 2500),
            "reso": None
        }
        ,
        "theta": {
            "range": (0, 0.6),
            "reso": 100
        },
        "phi": {
            "range": (0, 2 * np.pi),
            "reso": None
        }
    }

    Would give an image where one dimension is the X-coordinate and the other is
    the theta-coordinate

    :param mode: function to apply across each bin to get a value (default mean)
    :return: n by m matrix where each entry is a coordinate bin and corresponding
    value is given by mode applied to all data points in that bin.
    """
    ind_map = {
        "X": 1,
        "Y": 2,
        "theta": 4,
        "phi": 5
    }

    print(f"[1/2] binning data...")
    data = data.transpose()
    subset = (data[1] > params["X"]["range"][0]) * (data[1] < params["X"]["range"][1]) * \
             (data[2] > params["Y"]["range"][0]) * (data[2] < params["Y"]["range"][1]) * \
             (data[4] > params["theta"]["range"][0]) * (data[4] < params["theta"]["range"][1]) * \
             (data[5] > params["phi"]["range"][0]) * (data[5] < params["phi"]["range"][1])
    data = data.transpose()[subset].transpose()
    binned_data = {"E": obs[subset]}
    for key in tqdm(params.keys()):
        if params[key]["reso"]:
            bins = data[ind_map[key]] * params[key]["reso"]
            binned_data[key] = bins.astype('int')
    print("[2/2] creating image...")
    df = pd.DataFrame(binned_data)
    if mode == "mean":
        df_binned = df.groupby(list(binned_data.keys())[1:]).mean()
    elif mode == "std":
        df_binned = df.groupby(list(binned_data.keys())[1:]).std()
    elif mode == "counts":
        df_binned = df.groupby(list(binned_data.keys())[1:]).count()
    elif mode == "max":
        df_binned = df.groupby(list(binned_data.keys())[1:]).max()
    elif mode == "min":
        df_binned = df.groupby(list(binned_data.keys())[1:]).min()
    else:
        print(f"ERROR: parameter 'mode' must be either 'mean' or 'std'. Specified value: {mode}")
    E_by_bin = np.array(df_binned.E)
    return E_by_bin.reshape(
        [int(max(binned_data[key]) - min(binned_data[key]) + 1) for key in list(binned_data.keys())[1:]])

In [ ]:
def fft_filter(data, n_sigma=1, make_plots=False, make_sparse=False, no_filter=False):
    """
    Apply a fourier filter to physics data after the make_img preprocessing step.

    :param data: nxm matrix of binned data produced by the make_img function
    :param n_sigma: number of standard deviations to use in filtering (default 1)
    :param make_plots: whether to plot the filtered and unfiltered data for
    comparison (default False)
    :param make_sparse: whether to return the full filtered fourier transformed
    data or just the values that pass the filtering (default False)
    :param no_filter: whether to not perform filtering (default False)
    :return: filtered and fourier transformed version of data, if make_sparse=True
    this function will return a list of tuples that can be use to reconstruct the
    data later (see: load_from_sparse)
    """
    print(f"[1/2] performing fast Fourier transform along {len(data.shape)} axes...")
    data_fft = np.fft.fftn(data)
    if no_filter:
        return data_fft
    cutoff = np.mean(np.log(abs(data_fft.flatten()))) + n_sigma * np.std(np.log(abs(data_fft.flatten())))
    print(f"[2/2] filtering Fourier transformed data with cutoff e^{cutoff}")
    if make_sparse:
        saved_points = []
        for index, E in tqdm(np.ndenumerate(data_fft)):
            if np.log(abs(E)) > cutoff:
                saved_points.append((index, E))
        return saved_points
    else:
        for index, E in tqdm(np.ndenumerate(data_fft)):
            if np.log(abs(E)) < cutoff:
                data_fft[index] = 1 + 0j
            else:
                data_fft[index] = E
    if make_plots:
        plt.figure(figsize=(12, 8))
        plt.imshow(data, cmap='gray', origin="lower")
        plt.show()
        plt.figure(figsize=(12, 8))
        plt.imshow(abs(np.fft.ifftn(data_fft)), cmap='gray', origin="lower")
        plt.show()
    return data_fft